In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

from tqdm import tqdm
import time 
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

In [2]:
arr = sio.loadmat('traindata.mat')
images = (arr['I'])
images2=[]
for image in images:
    images2.append([image])
images2 = np.array(images2)
images2 = torch.FloatTensor(images2)
labels = torch.FloatTensor(arr['label'][0])
data = [images2, labels]
loader = [data]

In [3]:
transform = transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset,batch_size=200, shuffle=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=200, shuffle=False)

In [4]:
class CNN_modified(nn.Module):
  def __init__(self,size):
    super(CNN_modified, self).__init__()
    # Construct convolutional layer    
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(5,5))
    # num of input channel = 1, num of output channel 32, kernel size = 5×5 
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # Max-pooling with kernel size = 2×2, stride = 2
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5)
    # num of input channel = 1, num of output channel 32, kernel size = 5×5 (5 = 5×5)    
    self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
    # fc 쓰는 게 맞나??
    self.fc1 = nn.Linear(in_features=size, out_features=256)
    self.fc2 = nn.Linear(in_features=256, out_features=1) # The output size of last FC layer must be equal to num of class

  def forward(self, x):
    batchsize = x.size(0)
    print('batchsize= ',batchsize)
    x = self.conv1(x) # output for feature visuallization
    x = self.pool1(F.relu(x)) # Convolutional layer -> ReLU activation -> max_pooling
    x = self.conv2(x)
    x = self.pool2(F.relu(x))
    x = self.conv3(x)
    x = self.pool1(F.relu(x))
    print(x.shape)
    x = x.view(batchsize,-1) # Change the shape of output from [batch_size, channels, height, width] to [batch_size, channels*height*width]
    x = F.relu(self.fc1(x))
    out = self.fc2(x)
    return out

In [6]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # Construct convolutional layer    
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(5,5))
    # num of input channel = 1, num of output channel 32, kernel size = 5×5 
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # Max-pooling with kernel size = 2×2, stride = 2
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5)
    # num of input channel = 1, num of output channel 32, kernel size = 5×5 (5 = 5×5)    
    
    self.fc1 = nn.Linear(in_features=512, out_features=128)
    self.fc2 = nn.Linear(in_features=128, out_features=10) # The output size of last FC layer must be equal to num of class

  def forward(self, x):
    batchsize = x.size(0)
    x = self.conv1(x) # output for feature visuallization
    x = self.pool(F.relu(x)) # Convolutional layer -> ReLU activation -> max_pooling
    x = self.conv2(x)
    x = self.pool(F.relu(x))
    x = x.view(batchsize,-1) # Change the shape of output from [batch_size, channels, height, width] to [batch_size, channels*height*width]
    x = F.relu(self.fc1(x))
    out = self.fc2(x)
    return out

In [7]:
def train(model, n_epoch, loader, optimizer, criterion, device="cpu"):
  model.train()
  for epoch in tqdm(range(n_epoch)):
    running_loss = 0.0
    for i, data in enumerate(loader, 0):
      images, labels = data
      #print(images.size())
      #print('images\n',images)
      #print('labels\n',labels)
      images = images.to(device)
      #print('images.todevice',images)
      labels = labels.to(device)
      #print('labels.todevice',labels)
      optimizer.zero_grad()

      outputs = model(images)
      loss = criterion(input=outputs, target=labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    print('Epoch {}, loss = {:.3f}'.format(epoch, running_loss/len(loader)))
  print('Training Finished')


In [5]:
def train_modified(model, n_epoch, loader, optimizer, criterion, device="cpu"):
  model.train()
  for epoch in tqdm(range(n_epoch)):
    running_loss = 0.0
    for i, data in enumerate(loader, 0):
      images, labels = data
      labels = labels.type(torch.LongTensor)
      print(images.size())
      #print('images\n',images)
      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()

      outputs = model(images)
      loss = criterion(input=outputs, target=labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    print('Epoch {}, loss = {:.3f}'.format(epoch, running_loss/len(loader)))
  print('Training Finished')


In [6]:
def evaluate(model, loader, device="cpu"):
  model.eval()
  total=0
  correct=0
  with torch.no_grad():
    for data in loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted==labels).sum().item()
    
  acc = 100*correct/total
  return acc

In [7]:
#torch.cuda.empty_cache()
#PYTORCH_CUDA_ALLOC_CONF=5000 #max_split_size_mb
cnn_model = CNN_modified(2944).to("cuda")
optimizer = optim.SGD(params=cnn_model.parameters(), lr=0.01, momentum=0.9) #Ir 0.001 -> 0.01로 수정.
criterion = nn.CrossEntropyLoss()
train_modified(model=cnn_model, n_epoch=1, loader=loader, optimizer=optimizer, criterion=criterion, device="cuda")
#train(model=cnn_model, n_epoch=15, loader=trainloader, optimizer=optimizer, criterion=criterion, device="cuda")
acc = evaluate(cnn_model, testloader, device="cuda")
print('Test accuracy: {:.2f}%'.format(acc))

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

torch.Size([42, 1, 400, 42])
batchsize=  42
torch.Size([42, 64, 46, 1])


  0%|                                                                                            | 0/1 [00:03<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.